### Objective

Write an original ETL code that combines files and identifies cases that have a low blood pressure based on logic below. 

A successful project includes:

1. Original code file (Python)

2. Final report (excel or csv)

#### Import numpy and pandas for data manipulation

In [1]:
import pandas as pd
import numpy as np

#### Import visualization libraries and set %matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [11]:
# Read in the csv files as a dataframe called bloodpressure

bloodpressure = pd.read_csv('BloodPressures.csv', parse_dates=[1])

bloodpressure.head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE
0,123,2016-01-05 07:16:00,33.0
1,123,2016-01-05 07:17:00,75.0
2,123,2016-01-05 07:18:00,58.0
3,123,2016-01-05 07:19:00,93.0
4,123,2016-01-05 07:20:00,35.0


#### Investigating the bloodpressure dataframe

In [12]:
# Column names of the dataframe

print( 'Column Names:',bloodpressure.columns)

Column Names: Index(['PERSON_ID', 'TIME', 'SYSTOLIC_BLOOD_PRESSURE'], dtype='object')


In [13]:
# Dimension of the dataframe
print( 'Dimension:',bloodpressure.shape)

Dimension: (1577, 3)


In [14]:
## Check in the data type of each column
print('Data Type:',bloodpressure.dtypes)

Data Type: PERSON_ID                           int64
TIME                       datetime64[ns]
SYSTOLIC_BLOOD_PRESSURE           float64
dtype: object


In [15]:
# checking for null value in our across all columns in our dataframe

bloodpressure.isnull().sum()

PERSON_ID                  0
TIME                       0
SYSTOLIC_BLOOD_PRESSURE    1
dtype: int64

In [16]:
# The portion where I have error values in my dataframe

bloodpressure[bloodpressure.SYSTOLIC_BLOOD_PRESSURE.isnull()]

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE
1512,987,2016-10-17 18:40:00,NaN


Since the blood pressure thresholds are not clinically correct, and are created for this problem only, on this note I assumed that it will be approperiate to drop rows with Errors and null values Systolic BP.

In [17]:
# Droping PERSON_ID 1512 row with NaN value

bloodpressure.dropna(how='any', inplace=True)

In [18]:
# Checking for null values

bloodpressure.isnull().sum()

PERSON_ID                  0
TIME                       0
SYSTOLIC_BLOOD_PRESSURE    0
dtype: int64

In [28]:
# Converting SYSTOLIC_BLOOD_PRESSURE column from float data type to integer 

bloodpressure['SYSTOLIC_BLOOD_PRESSURE'] = bloodpressure.SYSTOLIC_BLOOD_PRESSURE.astype(int)

---

In [19]:
bloodpressure['TIME'].iloc[0]

Timestamp('2016-01-05 07:16:00')

In [22]:
time = bloodpressure['TIME'].iloc[0]
parse_dates=[1]
time.hour

7

In [24]:
time.minute

16

In [25]:
time.year

2016

In [26]:
# Lambda take "Time" column and return the hour, minutes, and day of week attribute
bloodpressure['Hour'] = bloodpressure['TIME'].apply(lambda time: time.hour)

bloodpressure['Minutes'] = bloodpressure['TIME'].apply(lambda time: time.minute)

bloodpressure['Day of week'] = bloodpressure['TIME'].apply(lambda time: time.dayofweek)

dmap = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}

bloodpressure['Day of week'] = bloodpressure['Day of week'].map(dmap)

In [29]:
bloodpressure.head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE,Hour,Minutes,Day of week
0,123,2016-01-05 07:16:00,33,7,16,Tue
1,123,2016-01-05 07:17:00,75,7,17,Tue
2,123,2016-01-05 07:18:00,58,7,18,Tue
3,123,2016-01-05 07:19:00,93,7,19,Tue
4,123,2016-01-05 07:20:00,35,7,20,Tue


In [30]:
bloodpressure.dtypes

PERSON_ID                           int64
TIME                       datetime64[ns]
SYSTOLIC_BLOOD_PRESSURE             int32
Hour                                int64
Minutes                             int64
Day of week                        object
dtype: object

In [31]:
bloodpressure.shape

(1576, 6)

--

In [37]:
# Read in the demographics csv files as a dataframe called demographics

demographics = pd.read_csv('Demographics.csv', parse_dates=[1])

demographics.head()

,PERSON_ID,SERVICE_DATE,AGE_MONTHS
0,123,2016-01-05,46
1,123,2016-02-13,47
2,456,2015-06-08,40
3,456,2015-08-15,42
4,456,2015-05-21,39


In [38]:
demographics.shape

(17, 3)

In [39]:
demographics.dtypes

PERSON_ID                int64
SERVICE_DATE    datetime64[ns]
AGE_MONTHS               int64
dtype: object

In [40]:
# Checking for null values

demographics.isnull().sum()

PERSON_ID       0
SERVICE_DATE    0
AGE_MONTHS      0
dtype: int64

In [41]:
# Join both dataframes using a left join

bloodpressure_demographics = pd.merge(bloodpressure, demographics, how='left')

bloodpressure_demographics.head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE,Hour,Minutes,Day of week,SERVICE_DATE,AGE_MONTHS
0,123,2016-01-05 07:16:00,33,7,16,Tue,2016-01-05,46
1,123,2016-01-05 07:16:00,33,7,16,Tue,2016-02-13,47
2,123,2016-01-05 07:17:00,75,7,17,Tue,2016-01-05,46
3,123,2016-01-05 07:17:00,75,7,17,Tue,2016-02-13,47
4,123,2016-01-05 07:18:00,58,7,18,Tue,2016-01-05,46


In [42]:
bloodpressure_demographics.shape

(4316, 8)

In [43]:
bloodpressure_demographics.isnull().sum()

PERSON_ID                  0
TIME                       0
SYSTOLIC_BLOOD_PRESSURE    0
Hour                       0
Minutes                    0
Day of week                0
SERVICE_DATE               0
AGE_MONTHS                 0
dtype: int64

---

In [44]:
# Exporting data to a database in SQL

from sqlalchemy import create_engine

import urllib


quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                     "Server=DESKTOP-H926MSV\SQLEXPRESS;"
                     "Database=BCH_Project;"
                     "Trusted_Connection=yes;")

engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

bloodpressure_demographics.to_sql('BP_DEMO', con = engine, if_exists = 'replace')

--

In [45]:
#################################################################################################################
##############################   Getting the file from SQL Server      ##########################################
#################################################################################################################
import pyodbc as podbc

conn2 = podbc.connect("Driver={SQL Server Native Client 11.0};"
                     "Server=DESKTOP-H926MSV\SQLEXPRESS;"
                     "Database=BCH_Project;"
                     "Trusted_Connection=yes;")

# rynning the SQL quey i need
SQL_Query = pd.read_sql_query('''SELECT * FROM BP_DEMO''', conn2)

#conn.close()



# Viz
SQL_Query.head()


,index,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE,Hour,Minutes,Day of week,SERVICE_DATE,AGE_MONTHS
0,0,123,2016-01-05 07:16:00,33,7,16,Tue,2016-01-05,46
1,1,123,2016-01-05 07:16:00,33,7,16,Tue,2016-02-13,47
2,2,123,2016-01-05 07:17:00,75,7,17,Tue,2016-01-05,46
3,3,123,2016-01-05 07:17:00,75,7,17,Tue,2016-02-13,47
4,4,123,2016-01-05 07:18:00,58,7,18,Tue,2016-01-05,46


1. Identify cases during which blood pressure dropped below the norm for the age (see below) for 14 continuous minutes or longer. Assume, that the PERSON_ID is the identifier for the patient, and SERVICE_DATE is the date of the surgery that they had. Surgeries don’t span over 1 day. The Age is given for that patient, for the surgery date.  The Blood pressure is only taken during the surgery duration. 

If the child reached 44 months, systolic blood pressure is considered low at 55 mmHg and below. Before 44 months of age, 46 mmHg and below is considered low. Since the patient is 40 months old at the time of the surgery, their threshold is 46 mmHg, hence, only row 1 would be considered low for 1 minute.

In [48]:
# Considering patients who are 40 months old and with SYSTOLIC_BLOOD_PRESSURE of less than or equal to 46

pd.read_sql_query('''SELECT PERSON_ID, SERVICE_DATE, COUNT(Minutes) AS duration_in_Min FROM BP_DEMO
WHERE Minutes >=14 AND SYSTOLIC_BLOOD_PRESSURE <=46 AND AGE_MONTHS = 40
GROUP BY PERSON_ID, AGE_MONTHS, SERVICE_DATE
ORDER BY PERSON_ID ASC''', conn2)

,PERSON_ID,SERVICE_DATE,duration_in_Min
0,456,2015-06-08,109
1,567,2019-05-10,24
